In [1]:
from math import *
import pandas as pd
import numpy as np
import statistics as st
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from scipy.interpolate import lagrange, interp1d
from pyproj import Transformer
import re
from leven import levenshtein
from sklearn.cluster import dbscan
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics
from shapely import geometry
import geopy.distance
import itertools
import warnings
warnings.filterwarnings('ignore')

#webscraping packages
import requests
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

#multiprocessing for faster webscraping
from multiprocessing.dummy import Pool

from datetime import datetime, timedelta
import difflib #for finding closest word match
import os
import ipdb as ipdb
%pdb off

Automatic pdb calling has been turned OFF


# 1. Compiling all 2019 CA flights into one dataframe

You only need to run section 1 once, as it saves the output into a csv file.

In [ ]:
def prepare_dataframe(file_path):
    #read in file and convert to dataframe
    if file_path[-3:] == 'xls':
        df = pd.read_html(file_path)[0]
        df.columns = df.columns.droplevel(0) #drop Multiindex header
        df = df[df.columns[:-1]] #drop last column
        flight_data = df[['ActualAirborneTime', 'ActualBlockTime','Carrier', 'FlightNumber', 'TailNumber', 'Departure', 'Arrival', 'AircraftType', 'ScheduledDepartureDate', 'TaxiOutTime', 'TaxiInTime', 'ActualWheelsOff', 'ActualWheelsOn']]
        return flight_data
    elif file_path[-3:] == 'csv':
        df = pd.read_csv(file_path, header = 1)
        col_names = list(df)
        col_names = [re.search('\w+(?=\\xa0)', word).group(0) for word in col_names]
        df.columns = col_names
        flight_data = df[['ActualAirborneTime', 'ActualBlockTime','Carrier', 'FlightNumber', 'TailNumber', 'Departure', 'Arrival', 'AircraftType', 'ScheduledDepartureDate', 'TaxiOutTime', 'TaxiInTime', 'ActualWheelsOff', 'ActualWheelsOn']]
        flight_data = flight_data.dropna()

        for col in list(flight_data):
            current_column = flight_data[col]
            flight_data[col] = [re.search('.+(?=\\xa0)', value).group(0) for value in current_column]
        return flight_data

**If compiling small airports, run section below:**

In [ ]:
#get folder of all airport-specific data directories
folder_name = 'ASPM_Data_Small'
directory_contents = os.listdir(folder_name)
directory_contents.remove('.DS_Store')
directory_contents.sort(key = str.upper)

#list all file path names for future looping
file_path_names = []
for aa in directory_contents:
    yy = os.listdir(folder_name + '/' + aa)
    for kk in yy:
        full_name = folder_name + '/' + aa + '/' + kk
        file_path_names.append(full_name)

**Else if compiling the 5 large airports (LAX, SFO, OAK, SJC, SAN), change folder name accordingly and run section below:**

In [ ]:
#for larger airports, run this code
folder_name = 'ASPM_Data_SAN'
file_contents = os.listdir(folder_name)
try:
    file_contents.remove('.DS_Store')
except:
    pass
file_contents.sort(key = str.upper)
file_path_names = []
for aa in file_contents:
    yy = folder_name + '/' + aa
    file_path_names.append(yy)

In [ ]:
#loop through all small airports to compile one dataframe of all flights
print(datetime.now())
flight_data_SAN = pd.DataFrame()
for file in file_path_names:
    print(file)
    xx = prepare_dataframe(file)
    flight_data_SAN = pd.concat([flight_data_SAN, xx], ignore_index = True).drop_duplicates()
print(datetime.now())

#save compiled flight data:
#flight_data_SAN.to_csv('Save Points/1/flight_data_SAN.csv')

**Run code below to combine all airport flight data into one dataframe:**

In [ ]:
small_airports_2019_df = pd.read_csv('Save Points/1/flight_data_small_airports.csv')
LAX_2019_df = pd.read_csv('Save Points/1/flight_data_LAX.csv')
SFO_2019_df = pd.read_csv('Save Points/1/flight_data_SFO.csv')
SAN_2019_df = pd.read_csv('Save Points/1/flight_data_SAN.csv')
SJC_2019_df = pd.read_csv('Save Points/1/flight_data_SJC.csv')
OAK_2019_df = pd.read_csv('Save Points/1/flight_data_OAK.csv')

all_CA_airports_2019_df = pd.concat([small_airports_2019_df, LAX_2019_df, SFO_2019_df, SAN_2019_df, 
                                     SJC_2019_df, OAK_2019_df], ignore_index = True).drop_duplicates()
all_CA_airports_2019_df = all_CA_airports_2019_df.drop(all_CA_airports_2019_df.columns[[0]], axis = 1)
#all_CA_airports_2019_df.to_csv('Save Points/1/all_CA_airports_2019.csv', index = False)

# 1.5 Reading in Saved Flight Data

In [2]:
f = pd.read_csv('airports.dat.txt', header = None)
f = f.rename(columns = {0: 'ID', 1: 'Name', 2: 'City', 3: 'Country', 4: 'IATA', 5: 'ICAO', 6: 'Latitude', 7: 'Longitude',
         8: 'Altitude', 9: 'Timezone', 10: 'DST', 11: 'Tz Database Timzone', 12: 'Type', 13: 'Source'})
f = f[f['Country'] == 'United States']
f = f[['IATA', 'ICAO', 'Latitude', 'Longitude']]

In [3]:
#creating two seperate dataframes: one df has intra-CA flights, the other OOC flights
all_CA_flights_2019 = pd.read_csv('Save Points/1/all_CA_airports_2019.csv')
#seperate international flights from domestic (we are only using domestic-CA flights)
all_US_CA_flights_2019 = all_CA_flights_2019[(all_CA_flights_2019['Departure'].isin(f['IATA'])) & (all_CA_flights_2019['Arrival'].isin(f['IATA']))]
ca_iata_codes = list(pd.read_csv('CA Geographical Data/CA_airports.csv')['AIRPORTID'])
intra_CA = all_US_CA_flights_2019[all_US_CA_flights_2019.Departure.isin(ca_iata_codes)]
intra_CA = intra_CA[intra_CA.Arrival.isin(ca_iata_codes)]
out_of_CA = all_US_CA_flights_2019[~all_US_CA_flights_2019.index.isin(intra_CA.index)]

In [ ]:
f

# 2. Straight-Line Method for Determining Portion of Flight Within CA

This section needs to be run only once, as the emissions fractions are saved as a csv file below.

In [ ]:
def lat_lon_merge(airport_df, loc_df):
    #merge on departures
    dep_merge = airport_df.merge(loc_df, left_on = 'Departure', right_on = 'IATA', how = 'left')
    dep_merge = dep_merge.drop(['IATA', 'ICAO'], axis = 1)
    dep_merge = dep_merge.rename(columns = {'Latitude': 'Dep Lat', 'Longitude': 'Dep Lon'})

    #merge on arrivals
    arr_merge = dep_merge.merge(loc_df, left_on = 'Arrival', right_on = 'IATA', how = 'left')
    arr_merge = arr_merge.drop(['IATA', 'ICAO'], axis = 1)
    arr_merge = arr_merge.rename(columns = {'Latitude': 'Arr Lat', 'Longitude': 'Arr Lon'})
    return arr_merge

In [ ]:
#read in data for CA vertices, define CA linear ring 
ca_vertices = pd.read_csv('CA Geographical Data/ca_vertices.csv').dropna()
x_coords = ca_vertices['xcoord']
y_coords = ca_vertices['ycoord']

ca_shape_points = [geometry.Point(i) for i in list(zip(x_coords, y_coords))]
ca_shape = geometry.polygon.LinearRing([[p.x, p.y] for p in ca_shape_points])

In [ ]:
#define function to create OD Pair Label
def compare_od_pairs(row):
    alph_od_pair = ''.join(sorted([row[5], row[6]]))
    return alph_od_pair

from pyproj import Proj, transform

#define function to convert lat/lon coordinates to x,y coordinates
def convert_to_xy(lon, lat):
    inProj  = Proj("+init=EPSG:4326")
    outProj = Proj("+init=EPSG:3857")
    return transform(inProj, outProj, lon, lat)

ca_iata_codes = list(pd.read_csv('CA Geographical Data/CA_airports.csv')['AIRPORTID'])

In [ ]:
with_locations = lat_lon_merge(out_of_CA, f)
insert = with_locations.apply(compare_od_pairs, axis = 1)
with_locations.insert(0, 'OD Pair Name', insert)
unique_OD = list(np.unique(np.array(with_locations['OD Pair Name'])))

In [ ]:
#this cell takes ~30-40 minutes to run; do not run if already run once (output saves to CSV below)
emissions_frac_within_CA = {}
for od_idx in unique_OD:
    first_airport = od_idx[0:3]
    second_airport = od_idx[3:6]
    f_lon, f_lat = f[f['IATA'] == first_airport]['Longitude'].values[0], f[f['IATA'] == first_airport]['Latitude'].values[0]
    s_lon, s_lat = f[f['IATA'] == second_airport]['Longitude'].values[0], f[f['IATA'] == second_airport]['Latitude'].values[0]
    f_pt = list(convert_to_xy(f_lon, f_lat))
    s_pt = list(convert_to_xy(s_lon, s_lat))
    travel_line = geometry.LineString([f_pt, s_pt])
    if travel_line.intersects(ca_shape):
        intersection = travel_line.intersection(ca_shape)
        if type(intersection) == geometry.multipoint.MultiPoint:
            intersection = intersection[0]
        if first_airport in ca_iata_codes:
            frac_within_ca = geometry.LineString([f_pt, intersection]).length / travel_line.length
        else:
            frac_within_ca = geometry.LineString([s_pt, intersection]).length / travel_line.length
        emissions_frac_within_CA[od_idx] = frac_within_ca
    else:
        emissions_frac_within_CA[od_idx] = np.nan

In [ ]:
frac_in_CA_df = pd.DataFrame.from_dict(emissions_frac_within_CA, orient = 'index').rename(columns = {0:'Frac in CA'})
frac_in_CA_df.to_csv('Save Points/2/frac_in_CA.csv')


In [ ]:
plt.figure(figsize = (5*1.5, 9*1.5))
x,y = ca_shape.xy
plt.plot(x, y, color = '#3c9efa')
plt.plot(*travel_line.xy, color= '#fca903')
plt.scatter(*intersection.xy, color = '#f74134', s = 100)
plt.scatter(*dep_point, color = '#f74134', s = 100)
plt.scatter(*arr_point, color = '#f74134', s = 100)
plt.show()

# 2.5 Appending OOC Emission Fractions and OD Pair Name

In [4]:
def od_pair_name_addition(df):
    a = df[['Departure', 'Arrival']].values
    a = np.sort(a)
    a_df = pd.DataFrame(a)
    a_df = pd.Series(a_df[0] + a_df[1])
    insert = pd.concat([a_df, pd.Series(df.index)], axis = 1).set_index(1)
    df_return = pd.concat([insert, df], axis = 1).rename(columns = {0: 'OD Pair Name'})
    return df_return

In [5]:
#adds OD pair name for identification to each dataframe
intra_CA = od_pair_name_addition(intra_CA)
out_of_CA = od_pair_name_addition(out_of_CA)

In [6]:
frac_in_CA_df = pd.read_csv('Save Points/2/frac_in_CA.csv')
frac_in_CA_df = frac_in_CA_df.rename(columns = {'Unnamed: 0': 'OD Pair Name'})
out_of_CA = frac_in_CA_df.merge(out_of_CA, on = 'OD Pair Name', how = 'left')

# 3. Using European Environmental Agency (EEA) Aviation Master Emissions Calculator to calculate CCD (Cruise/Climb/Descent) Fuel Flow and Emissions Indices 

Can be found at: https://www.eea.europa.eu/publications/emep-eea-guidebook-2019/part-b-sectoral-guidance-chapters/1-energy/1-a-combustion/1-a-3-a-aviation-1/view

**The following lines of code need only be run once (saves to csv below):**

In [ ]:
def lat_lon_merge(airport_df, loc_df):
    #merge on departures
    dep_merge = airport_df.merge(loc_df, left_on = 'Departure', right_on = 'IATA', how = 'left')
    dep_merge = dep_merge.drop(['IATA', 'ICAO'], axis = 1)
    dep_merge = dep_merge.rename(columns = {'Latitude': 'Dep Lat', 'Longitude': 'Dep Lon'})

    #merge on arrivals
    arr_merge = dep_merge.merge(loc_df, left_on = 'Arrival', right_on = 'IATA', how = 'left')
    arr_merge = arr_merge.drop(['IATA', 'ICAO'], axis = 1)
    arr_merge = arr_merge.rename(columns = {'Latitude': 'Arr Lat', 'Longitude': 'Arr Lon'})
    return arr_merge

In [ ]:
#following four lines gets rid of flights in which A/C Type is not given:
intra_CA = intra_CA[~intra_CA['AircraftType'].isnull()]
out_of_CA = out_of_CA[~out_of_CA['AircraftType'].isnull()]

intra_CA = intra_CA[~(intra_CA['AircraftType'] == '-1')]
out_of_CA = out_of_CA[~(out_of_CA['AircraftType'] == '-1')]

orig_len_intra = len(intra_CA); orig_len_ooc = len(out_of_CA)

intra_CA_w_loc = lat_lon_merge(intra_CA, f)
out_of_CA_w_loc = lat_lon_merge(out_of_CA, f)

In [ ]:
#calculate distance of each flight (in Nautical-Miles (NM))
from pyproj import Geod

wgs84_geod = Geod(ellps='WGS84') #Distance will be measured on this ellipsoid - more accurate than a spherical method

#Get distance between pairs of lat-lon points
def Distance(lat1,lon1,lat2,lon2):
    az12, az21,dist = wgs84_geod.inv(lon1,lat1,lon2,lat2) #Yes, this order is correct
    return dist

distances_i = Distance(intra_CA_w_loc['Dep Lat'].tolist(), intra_CA_w_loc['Dep Lon'].tolist(), intra_CA_w_loc['Arr Lat'].tolist(), intra_CA_w_loc['Arr Lon'].tolist())
distances_o = Distance(out_of_CA_w_loc['Dep Lat'].tolist(), out_of_CA_w_loc['Dep Lon'].tolist(), out_of_CA_w_loc['Arr Lat'].tolist(), out_of_CA_w_loc['Arr Lon'].tolist())

distances_i_nm = np.array(distances_i) / 1852
distances_o_nm = np.array(distances_o) / 1852

intra_CA['Distance (NM)'] = distances_i_nm
out_of_CA['Distance (NM)'] = distances_o_nm

In [ ]:
#read in EI/FF data and clean dataframe for calculations
aircraft_master_CCD_emissions = pd.read_csv('aircraft_master_CCD_emissions.csv')
aircraft_master_CCD_emissions = aircraft_master_CCD_emissions[aircraft_master_CCD_emissions['LTO or CCD'] == 'CCD']

aircraft_master_CCD_emissions = aircraft_master_CCD_emissions[['IMPACT ACFT ID', '\xa0DURATION', 
                                                               '\xa0DISTANCE NM', '\xa0FUEL BURNT KG', 
                                                               'CO2 (3,15 or 3,05)\xa0', '\xa0NOX', 
                                                               '\xa0CO', '\xa0HC', '\xa0PM TOTAL']]
col_names = list(aircraft_master_CCD_emissions)
col_names_new = [re.sub("[\xa0]", "", cc) for cc in col_names]
aircraft_master_CCD_emissions.columns = col_names_new
aircraft_master_CCD_emissions = aircraft_master_CCD_emissions.replace({"\\xa0": "", " ": ""}, regex = True)
aircraft_master_CCD_emissions = aircraft_master_CCD_emissions.astype({'DISTANCE NM': 'float64', 'FUEL BURNT KG': 'float64',
                                                                      'CO2 (3,15 or 3,05)': 'float64', 'NOX': 'float64',
                                                                      'CO': 'float64', 'HC': 'float64', 'PM TOTAL': 'float64'})


In [ ]:
#calculate emissions indices for each pollutant
fuel_consumed_raw = aircraft_master_CCD_emissions['FUEL BURNT KG']
CO2_cr_ei = aircraft_master_CCD_emissions['CO2 (3,15 or 3,05)'] / fuel_consumed_raw #leave in kg
NOx_cr_ei = aircraft_master_CCD_emissions['NOX'] * 1000 / fuel_consumed_raw #convert rest to grams
CO_cr_ei = aircraft_master_CCD_emissions['CO'] * 1000 / fuel_consumed_raw
HC_cr_ei = aircraft_master_CCD_emissions['HC'] * 1000 / fuel_consumed_raw
PM_cr_ei = aircraft_master_CCD_emissions['PM TOTAL'] * 1000 / fuel_consumed_raw #kg/kg --> g/kg

#calculate fuel flow for cruising (alt. version of ff with pressure/temp coeff)
duration_sec = [sum(x * int(t) for x, t in zip([1, 60, 3600], reversed(tt.split(":")))) for tt in list(aircraft_master_CCD_emissions['DURATION'])]
ff_cr_alt = fuel_consumed_raw / duration_sec  

In [ ]:
#create new dataframe with EI and FF values calculated above
ac_cruise_ff_ei = pd.DataFrame()
ac_cruise_ff_ei['AircraftType'] = aircraft_master_CCD_emissions['IMPACT ACFT ID']
ac_cruise_ff_ei['Distance (NM)'] = aircraft_master_CCD_emissions['DISTANCE NM']
ac_cruise_ff_ei['Fuel Flow CCD (kg/sec)'] = ff_cr_alt
ac_cruise_ff_ei['HC EI CCD (g/kg)'] = HC_cr_ei
ac_cruise_ff_ei['CO EI CCD (g/kg)'] = CO_cr_ei
ac_cruise_ff_ei['NOx EI CCD (g/kg)'] = NOx_cr_ei
ac_cruise_ff_ei['PM EI CCD (g/kg)'] = PM_cr_ei
ac_cruise_ff_ei['CO2 EI CCD (kg/kg)'] = CO2_cr_ei
ac_cruise_ff_ei.to_csv('ac_cruise_ff_ei.csv')

In [ ]:
#for each unique aircraft type: create interpolation functions that relate distance of flight to FF and EI
ac_unique = list(ac_cruise_ff_ei.AircraftType.unique())
ff_interp = {}; hc_ei_interp = {}; co_ei_interp = {}; nox_ei_interp = {}; pm_ei_interp = {}; co2_ei_interp = {}
for ac in ac_unique:
    unique_ac_df = ac_cruise_ff_ei[ac_cruise_ff_ei['AircraftType'] == ac]
    x = unique_ac_df['Distance (NM)']
    ff_interp[ac] = interp1d(x, unique_ac_df['Fuel Flow CCD (kg/sec)'], fill_value = 'extrapolate')
    hc_ei_interp[ac] = interp1d(x, unique_ac_df['HC EI CCD (g/kg)'], fill_value = 'extrapolate')
    co_ei_interp[ac] = interp1d(x, unique_ac_df['CO EI CCD (g/kg)'], fill_value = 'extrapolate')
    nox_ei_interp[ac] = interp1d(x, unique_ac_df['NOx EI CCD (g/kg)'], fill_value = 'extrapolate')
    pm_ei_interp[ac] = interp1d(x, unique_ac_df['PM EI CCD (g/kg)'],fill_value = 'extrapolate')
    co2_ei_interp[ac] = interp1d(x, unique_ac_df['CO2 EI CCD (kg/kg)'], fill_value = 'extrapolate')

In [ ]:
intra_CA

In [ ]:
set1 = set(ac_cruise_ff_ei['AircraftType'].unique())
set2 = set(all_US_CA_flights_2019.AircraftType.unique())
missing_ac = set2 - set1
not_missing_ac = set1.intersection(set2)
all_ac_data_provided = list(ac_cruise_ff_ei.AircraftType.unique())

In [ ]:
pd.set_option('display.max_rows', 500)
missing_ac_df = all_US_CA_flights_2019[all_US_CA_flights_2019['AircraftType'].isin(missing_ac)]
#missing_ac_df.groupby('AircraftType').size()

In [ ]:
alt_ac_dict = {'A20N': 'A320', 'A21N': 'A321', 'A330': 'A333', 'A359': 'A380', 'A388': 'A380', 'ASTR': 'G150', 
               'AT43': 'ATR42', 'AT45': 'ATR42', 'AT72': 'ATR72', 'B190': 'BE90', 'B350': 'BE300', 'B38M': 'B737', 
               'B39M': 'B737', 'B712': 'MD90', 'B757': 'B573', 'B767': 'B764', 'B77L': 'B772', 'B777': 'B773', 
               'B78X': 'B788', 'BE20': 'BE200', 'BE30': 'BE300', 'BE40': 'BE400', 'BE9L': 'BE90', 'BE9T': 'BE90',
               'C25A': 'C525', 'C25B': 'C525', 'C25C': 'C525', 'C25M': 'C525', 'C56X': 'C560', 'C68A': 'C680', 
               'C68L': 'C680', 'CL30': 'CL300', 'CL35': 'CL300', 'CL60': 'CL600', 'CRJ1': 'CL600RJ',
               'CRJ2': 'CL600RJ', 'CRJ7': 'CL700RJ', 'CRJ9': 'CL900RJ', 'DH8': 'DHC8', 'DH8A': 'DHC8',
               'DH8B': 'DHC8', 'DH8C': 'DHC8', 'DH8D': 'DHC8', 'E35L': 'EMB600', 'E45X': 'E145', 'E50P': 'EMB100',
               'E545': 'E500', 'E545': 'E500', 'E55P': 'EMB300', 'E75L': 'E175', 'E75S': 'E175', 'F2TH': 'FA2000',
               'F900': 'FA900', 'GA5C': 'GA7', 'GA6C': 'GA7', 'GALX': 'G200', 'GL5T': 'BD700', 'GLEX': 'BD700',
               'GLF4': 'G400', 'GLF5': 'G500', 'GLF6': 'G650', 'H25A': 'BAE125', 'H25B': 'BAE125', 
               'H25C': 'BAE125', 'MD83': 'MD80', 'MD87': 'MD80', 'MD88': 'MD80', 'SW4': 'SA226'} 

In [ ]:
#replace missing AircraftType with equivalent or approximations as given in alt_ac_dict
intra_mapping = intra_CA['AircraftType'].map(alt_ac_dict, na_action = 'ignore').dropna()
ooc_mapping = out_of_CA['AircraftType'].map(alt_ac_dict, na_action = 'ignore').dropna()

intra_CA.loc[intra_mapping.index, 'AircraftType'] = intra_mapping
out_of_CA.loc[ooc_mapping.index, 'AircraftType'] = ooc_mapping

In [ ]:
intra_CA = intra_CA[intra_CA['AircraftType'].isin(all_ac_data_provided)]
out_of_CA = out_of_CA[out_of_CA['AircraftType'].isin(all_ac_data_provided)]

loss_ac_type_intra = (orig_len_intra - len(intra_CA)) / orig_len_intra
loss_ac_type_ooc = (orig_len_ooc - len(out_of_CA)) / orig_len_ooc
print('Loss due to missing A/C Type (Intra-CA): {}%'.format(round(loss_ac_type_intra * 100, 2)))
print('Loss due to missing A/C Type (OOC): {}%'.format(round(loss_ac_type_ooc * 100, 2)))

In [ ]:
def calculate_ff_ei_ccd(df):
    print(datetime.now())
    df['Fuel Flow CCD (kg/sec)'] = df.apply(lambda x: ff_interp[x['AircraftType']](x['Distance (NM)']), axis = 1)
    print(datetime.now())
    df['HC EI CCD (g/kg)'] = df.apply(lambda x: hc_ei_interp[x['AircraftType']](x['Distance (NM)']), axis = 1)
    df['CO EI CCD (g/kg)'] = df.apply(lambda x: co_ei_interp[x['AircraftType']](x['Distance (NM)']), axis = 1)
    df['NOx EI CCD (g/kg)'] = df.apply(lambda x: nox_ei_interp[x['AircraftType']](x['Distance (NM)']), axis = 1)
    df['PM EI CCD (g/kg)'] = df.apply(lambda x: pm_ei_interp[x['AircraftType']](x['Distance (NM)']), axis = 1)
    df['CO2 EI CCD (kg/kg)'] = df.apply(lambda x: co2_ei_interp[x['AircraftType']](x['Distance (NM)']), axis = 1)
    print(datetime.now())
    return df

In [ ]:
intra_CA = calculate_ff_ei_ccd(intra_CA)
out_of_CA = calculate_ff_ei_ccd(out_of_CA)

In [ ]:
intra_CA.to_csv('Save Points/3/intra_CA_CCD.csv', index = False)
out_of_CA.to_csv('Save Points/3/out_of_CA_CCD.csv',index = False)

# 4. Preparing N-Number and LTO FF/EI Dataframes

In [7]:
#reading in entire database of aircraft N-Number and Engine Mfr Code
aircraft_data_master = pd.read_csv('ReleasableAircraft/MASTER.txt')[['N-NUMBER', 'ENG MFR MDL']]
aircraft_data_master = aircraft_data_master[aircraft_data_master['ENG MFR MDL'] != '     ']
aircraft_data_master['ENG MFR MDL'] = aircraft_data_master['ENG MFR MDL'].astype(int)

#reading in Engine database (Engine Mfr Code and Model Name)
aircraft_data_engine = pd.read_csv('ReleasableAircraft/ENGINE.txt')[['CODE', 'MODEL']]
aircraft_data_engine = aircraft_data_engine.rename(columns = {'CODE': 'ENG MFR MDL'})

#merge two databases ==> relates N-Number (tail number) of aircraft to Model Name to find fuel flow later
aircraft_model_names = aircraft_data_master.merge(aircraft_data_engine, on = 'ENG MFR MDL', how = 'left')
aircraft_model_names['N-NUMBER'] = 'N' + aircraft_model_names['N-NUMBER'].astype(str)
aircraft_model_names = aircraft_model_names.rename(columns = {'N-NUMBER': 'TailNumber'})
aircraft_model_names = aircraft_model_names.set_index('TailNumber')
aircraft_model_names.index = [index.strip() for index in aircraft_model_names.index]

In [8]:
# for guest lecture:
#aircraft_model_names.to_csv('ReleasableAircraft/nn_em.csv')

In [10]:
#reading in fuel flow and emissions index data
fuel_flow_emissions = pd.read_excel('Emissions Data/emissions_indices.xlsx').rename(columns = {'Engine Identification': 'MODEL'})
fuel_flow_emissions = fuel_flow_emissions[['MODEL', 'Fuel Flow T/O (kg/sec)', 'Fuel Flow C/O (kg/sec)', 'Fuel Flow App (kg/sec)', 'Fuel Flow Idle (kg/sec)',
                                           'HC EI T/O (g/kg)', 'HC EI C/O (g/kg)', 'HC EI App (g/kg)', 'HC EI Idle (g/kg)',
                                           'CO EI T/O (g/kg)', 'CO EI C/O (g/kg)', 'CO EI App (g/kg)', 'CO EI Idle (g/kg)',
                                           'NOx EI T/O (g/kg)', 'NOx EI C/O (g/kg)', 'NOx EI App (g/kg)', 'NOx EI Idle (g/kg)', 
                                           'SN T/O', 'SN C/O', 'SN App', 'SN Idle']]
fuel_flow_emissions = fuel_flow_emissions.dropna()
fuel_flow_emissions = fuel_flow_emissions.groupby('MODEL', as_index=False).mean()
fuel_flow_emissions = fuel_flow_emissions.set_index('MODEL')
fuel_flow_emissions = fuel_flow_emissions.rename(columns={'SN T/O': 'SN EI T/O', 'SN C/O': 'SN EI C/O', 'SN App': 'SN EI App', 'SN Idle': 'SN EI Idle'})

# 5. Calculating Fuel Emissions

In [11]:
intra_CA = pd.read_csv('Save Points/3/intra_CA_CCD.csv')
out_of_CA = pd.read_csv('Save Points/3/out_of_CA_CCD.csv')

In [12]:
ca_airport_codes = pd.read_csv('CA Geographical Data/CA_airports.csv')['AIRPORTID']
out_of_CA['From CA'] = out_of_CA['Departure'].isin(ca_airport_codes).astype(int)

In [13]:
unique_aircrafts = np.unique(list(out_of_CA['AircraftType']) +  list(intra_CA['AircraftType']))
print(len(unique_aircrafts))

101


In [14]:
no_engines_dict = {'A306': 2, 'A319': 2, 'A310': 2, 'A320': 2, 'A332': 2, 'A333': 2, 'A343': 4, 'A346': 4,
                   'A321': 2, 'A380': 4, 'AN12': 4, 'ATR42': 2, 'ATR72': 2, 'B722': 3, 'B733': 2,
                   'B734': 2, 'B732': 2, 'B735': 2, 'B737': 2, 'B738': 2, 'B739': 2, 'B744': 4, 'B748': 4, 
                   'B752': 2, 'B753': 2, 'B762': 2, 'B763': 2, 'B764': 2, 'B772': 2, 'B773': 2, 'B77W': 2, 
                   'B788': 2, 'B789': 2, 'BAE125': 2, 'BD700': 2, 'BE200': 2, 'BE300': 2, 'BE400': 2, 'BE55': 2, 
                   'BE58': 2, 'BE90': 2, 'BE99': 2, 'C208': 1, 'C340': 2, 'C402': 2, 'C414': 2, 'C421': 2,
                   'C425': 2, 'C501': 2, 'C510': 2, 'C525': 2, 'C550': 2, 'C551': 2, 'C560': 2, 'C650': 2, 
                   'C680': 2, 'C750': 2, 'CL300': 2, 'CL600': 2, 'CL600RJ': 2, 'CL700RJ': 2, 'CL900RJ': 2,
                   'DC10': 3, 'DHC6': 2, 'DHC8': 2, 'E120': 2, 'E135': 2, 'E145': 2, 'E170': 2, 'E175': 2, 
                   'E500': 1, 'EMB100': 2, 'EMB300': 2, 'EMB600': 2, 'FA10': 2, 'FA20': 2, 'FA2000': 2, 
                   'FA50': 3, 'FA7X': 3, 'FA900': 3, 'G150': 2, 'G200': 1, 'G280': 2, 'G400': 2, 'G500': 2, 
                   'G650': 2, 'GA7': 2, 'LJ31': 2, 'LJ35': 2, 'LJ40': 2, 'LJ45': 2, 'LJ55': 2, 'LJ60': 2, 
                   'LJ75': 2, 'MD11': 3, 'MD80': 2, 'MD90': 2, 'PA31': 2, 'PC12': 1, 'PRM1': 2, 'SA226': 2}

In [15]:
intra_CA['No. Engines'] = intra_CA['AircraftType'].map(no_engines_dict)
out_of_CA['No. Engines'] = out_of_CA['AircraftType'].map(no_engines_dict)

In [17]:
# weather 

#Los Angeles County:
wi_l_la = np.mean([51, 51, 51])
wi_h_la = np.mean([67, 67, 67])
sp_l_la = np.mean([51, 53, 56])
sp_h_la = np.mean([67, 69, 70])
su_l_la = np.mean([58, 62, 62])
su_h_la = np.mean([73, 77, 79])
fa_l_la = np.mean([62, 59, 55])
fa_h_la = np.mean([78, 75, 71])

#Sacremento:
wi_l_sa = np.mean([38, 39, 41])
wi_h_sa = np.mean([54, 48, 60])
sp_l_sa = np.mean([44, 46, 51])
sp_h_sa = np.mean([65, 71, 80])
su_l_sa = np.mean([56, 58, 58])
su_h_sa = np.mean([87, 92, 91])
fa_l_sa = np.mean([56, 50, 43])
fa_h_sa = np.mean([87, 78, 64])

#Eureka:
wi_l_eu = np.mean([41, 41, 42])
wi_h_eu = np.mean([55, 56, 56])
sp_l_eu = np.mean([43, 44, 47])
sp_h_eu = np.mean([57, 58, 60])
su_l_eu = np.mean([50, 52, 53])
su_h_eu = np.mean([62, 53, 64])
fa_l_eu = np.mean([50, 47, 44])
fa_h_eu = np.mean([64, 62, 58])

#San Diego:
wi_l_sd = np.mean([49, 48, 65])
wi_h_sd = np.mean([65, 65, 76])
sp_l_sd = np.mean([61, 54, 67])
sp_h_sd = np.mean([73, 69, 76])
su_l_sd = np.mean([65, 62, 59])
su_h_sd = np.mean([75, 71, 69])
fa_l_sd = np.mean([56, 53, 51])
fa_h_sd = np.mean([67, 66, 65])

#San Francisco County:
wi_l_sf = np.mean([46, 53, 54])
wi_h_sf = np.mean([57, 66, 67])
sp_l_sf = np.mean([51, 49, 49])
sp_h_sf = np.mean([64, 63, 62])
su_l_sf = np.mean([47, 46, 55])
su_h_sf = np.mean([60, 57, 68])
fa_l_sf = np.mean([55, 54, 50])
fa_h_sf = np.mean([70, 69, 63])

#Fresno:
wi_l_fr = np.mean([38, 38, 42])
wi_h_fr = np.mean([56, 56, 63])
sp_l_fr = np.mean([45, 49, 55])
sp_h_fr = np.mean([68, 75, 84])
su_l_fr = np.mean([61, 66, 65])
su_h_fr = np.mean([92, 96, 96])
fa_l_fr = np.mean([61, 53, 43])
fa_h_fr = np.mean([91, 80, 66])

wi_l = (wi_l_la + wi_l_sa + wi_l_eu + wi_l_sd + wi_l_sf + wi_l_fr) / 6
wi_h = (wi_h_la + wi_h_sa + wi_h_eu + wi_h_sd + wi_h_sf + wi_h_fr) / 6
sp_l = (sp_l_la + sp_l_sa + sp_l_eu + sp_l_sd + sp_l_sf + sp_l_fr) / 6
sp_h = (sp_h_la + sp_h_sa + sp_h_eu + sp_h_sd + sp_h_sf + sp_h_fr) / 6
su_l = (su_l_la + su_l_sa + su_l_eu + su_l_sd + su_l_sf + su_l_fr) / 6
su_h = (su_h_la + su_h_sa + su_h_eu + su_h_sd + su_h_sf + su_h_fr) / 6
fa_l = (fa_l_la + fa_l_sa + fa_l_eu + fa_l_sd + fa_l_sf + fa_l_fr) / 6
fa_h = (fa_h_la + fa_h_sa + fa_h_eu + fa_h_sd + fa_h_sf + fa_h_fr) / 6 


In [18]:
#for Guest Lecture!

weather_df = pd.DataFrame()
weather_df['Low'] = [wi_l, sp_l, su_l, fa_l]
weather_df['High'] = [wi_h, sp_h, su_h, fa_h]
weather_df['Season'] = ['Winter', 'Spring', 'Summer', 'Fall']
weather_df = weather_df.set_index('Season')
weather_df.to_csv('Guest Lecture/weather_data.csv')

In [19]:
#taken from: https://stackoverflow.com/questions/11384714/ignore-case-with-difflib-get-close-matches
def get_close_matches_icase(word, possibilities, *args, **kwargs):
    """ Case-insensitive version of difflib.get_close_matches """
    lword = word.lower()
    lpos = {}
    for p in possibilities:
        if p.lower() not in lpos:
            lpos[p.lower()] = [p]
        else:
            lpos[p.lower()].append(p)
    lmatches = difflib.get_close_matches(lword, lpos.keys(), *args, **kwargs)
    ret = [lpos[m] for m in lmatches]
    ret = itertools.chain.from_iterable(ret)
    return set(ret)

In [20]:
#hand-created dictionary of engine approximations based on aircraft types, based on ICAO's 2021
#Aricraft Emissions Databank
engine_approx = {'A306': 'CF6 SERIES', 'A310': 'CF6 SERIES', 'A319': 'CFM56 SERIES', 'A320':'CFM56-5B4/P', 
                 'A321': 'V2533-A5', 'A332': 'CF6-80E1 SERIES', 'A333':'CF6-80E1 SERIES', 'A343': 'CFM56-5C4/P',
                 'A346': 'Trent 556', 'A380':'Trent 970 SERIES', 'A388': 'Trent 970', 'ATR72': 'PW100 SERIES',
                 'B722': 'JT8D-11', 'B732': 'JT8D-15 SERIES', 'B733': 'CFM56-3B SERIES', 'B734': 'CFM56-3C SERIES',
                 'B735': 'CFM56-3B SERIES', 'B736': 'CFM56-7B20', 'B737': 'CFM56-7B SERIES', 
                 'B738': 'CFM56-7B SERIES', 'B739': 'CFM56-7B SERIES', 'B744': 'RB211-524H', 'B748': 'GEnx-2B67', 
                 'B752': 'RB211-535E4', 'B753': 'PW2037', 'B762': 'PW4062', 'B763': 'PW4062', 'B764': 'PW4062',
                 'B772': 'GE90-94B', 'B773': 'PW4098', 'B77W': 'GE90-115B', 'B788': 'GEnx-1B SERIES', 
                 'B789':'GEnx-1B SERIES', 'BAE125': 'TFE731-3', 'BD700':'BR700-710 SERIES', 
                 'BE400': 'JT15D-5 SERIES', 'C501': 'JT15D-1 series', 'C550': 'JT15D-4 series', 
                 'C650': 'TFE731-3', 'C680': 'PW306A', 'C750': 'AE3007C1', 'CL300': 'HTF7000 SERIES', 
                 'CL600': 'CF34-3A SERIES', 'CL600RJ': 'CF34-3B SERIES', 'CL700RJ': 'CF34-8C SERIES', 
                 'CL900RJ': 'CF34-8C5 SERIES', 'CRJ9': 'CF34-8C5', 'DC10': 'CF6-6D SERIES', 
                 'E135': 'AE3007A3', 'E145': 'AE3007A SERIES', 'E170': 'CF34-8E SERIES', 
                 'E190': 'CF34-10E SERIES', 'FA10': 'TFE731-2-2B SERIES', 'FA50': 'TFE731-3', 'FA7X': 'PW307A', 
                 'FA900': 'TFE731-3', 'F100': 'TAY 650', 'G200': 'PW306A', 'G400': 'TAY 611-8C', 
                 'G500': 'PW814GA', 'G650': 'PW815GA', 'LJ31': 'TFE731-2-2B', 'LJ35': 'TFE731-2-2B', 
                 'LJ40': 'TFE731 SERIES', 'LJ55': 'TFE731-3', 'MD11': 'CF6-80C2 SERIES', 'MD80': 'JT8D SERIES', 
                 'MD90': 'V2525-D5', 'RJ1H': 'LF507-1F'}

In [21]:
def calculate_fuel_emissions_fulldf(flight_data):        
    print(datetime.now())
    #first merge: merging flight data with engine types for each flight 
    dfv1 = flight_data.merge(aircraft_model_names, left_on = 'TailNumber', right_index = True, how = 'left')
    dfv1 = dfv1.drop('ENG MFR MDL', axis = 1);
    
    missing = dfv1[(dfv1['MODEL'].isnull()) & ~(dfv1['AircraftType'].isnull())] #only engine model is missing; aircraft type is not missing
    not_missing = dfv1[~(dfv1['AircraftType'].isnull()) & ~(dfv1['MODEL'].isnull())] #both engine model and ac type are not misisng 
    not_missing['MODEL'] = not_missing['MODEL'].str.strip()
    missing.index = missing.index.astype(int); not_missing.index = not_missing.index.astype(int)
    
    #creating dictionary matching aircraft type to engine models
    ac_type_model = not_missing[['AircraftType', 'MODEL']].drop_duplicates()
    ac_type_model = ac_type_model.dropna()
    drop_indices = ac_type_model[(ac_type_model['AircraftType'].str.len() != 4) & (ac_type_model['AircraftType'].str.len() != 3)].index
    ac_type_model = ac_type_model.drop(drop_indices)
    ac_types = [i[0] for i in ac_type_model.groupby('AircraftType')]
    
    #for loop below runs through present aircraft types and determines most likely engine pairing based on
    #density based clustering algorithm using levenshtein distance between strings
    ac_type_engine_dict = {}
    for aa in ac_types:
        matching_engines = list(ac_type_model[ac_type_model['AircraftType'] == aa]['MODEL'])
        try:
            #defining levenshtein distance for list of strings
            def lev_metric(x, y):
                i, j = int(x[0]), int(y[0])     # extract indices
                return levenshtein(matching_engines[i], matching_engines[j])
            X = np.arange(len(matching_engines)).reshape(-1, 1)
            grouping = list(dbscan(X, metric=lev_metric, eps=9, min_samples=2, algorithm='brute')[1])
            mode = matching_engines[grouping.index(st.mode(grouping))]
            approx_ac_type = re.search(r'.*(?=-)|.*(?=\s)', mode).group(0) + ' SERIES'
        except:
            mode = sorted(matching_engines)[0]
            try:
                approx_ac_type = re.search(r'.*(?=-)|.*(?=\s)', mode).group(0) + ' SERIES'
            except:
                approx_ac_type = mode + ' SERIES'
        ac_type_engine_dict[aa] = approx_ac_type
    
    #maps ac type to engine 
    mapping = missing['AircraftType'].map(ac_type_engine_dict, na_action='ignore')
    mapping = mapping.dropna()
    dfv1.loc[mapping.index, 'MODEL'] = mapping
    dfv1['MODEL'] = dfv1['MODEL'].str.strip()
    dfv1.index = dfv1.index.astype(int)
    
    still_missing = dfv1[dfv1['MODEL'].isnull()]
    still_missing_mapping = still_missing['AircraftType'].map(engine_approx, na_action = 'ignore')
    still_missing_mapping = still_missing_mapping.dropna()
    
    dfv1.loc[still_missing.index, 'MODEL'] = still_missing_mapping
    dfv1 = dfv1.dropna()
    dfv1['MODEL'] = dfv1['MODEL'].str.strip()
    dfv1.index = dfv1.index.astype(int)
    
    print('First merge made and approximations subsituted')
    first_loss = str(round(100 * (len(flight_data) - len(dfv1))/len(flight_data), 2))
    print('Data lost at 1st merge: ' + first_loss + '%')
    
    #create approximations of fuel flow for engine models not listed in fuel_flow_df
    missing_groups = list(set(dfv1['MODEL']) - set(fuel_flow_emissions.index))
    for kk in missing_groups:
        if 'SERIES' in kk:
            series_name = kk.replace(' SERIES', '')
            closest_matches = get_close_matches_icase(series_name, fuel_flow_emissions.index)
            approx_fuel_flow = fuel_flow_emissions.loc[closest_matches].mean(axis = 0) #returns a pd.Series 
            new_row = pd.concat([pd.Series([kk]), approx_fuel_flow])
            fuel_flow_emissions.loc[kk] = new_row
        else:
            closest_matches = get_close_matches_icase(kk, fuel_flow_emissions.index)
            approx_fuel_flow = fuel_flow_emissions.loc[closest_matches].mean(axis = 0) #returns a pd.Series 
            new_row = pd.concat([pd.Series([kk]), approx_fuel_flow])
            fuel_flow_emissions.loc[kk] = new_row
    print('Approximations for missing engine models made...')

    #second merge: merging dfv1 with fuel flow emissions by engine model
    dfv2 = dfv1.merge(fuel_flow_emissions, on = 'MODEL', how = 'left')
    dfv2 = dfv2.dropna()
    print('Second merge made...')
    second_loss = str(round((len(dfv1) - len(dfv2))/len(flight_data),2)*100)
    print('Data lost at 2nd merge: ' + second_loss + '%')
    
    month = dfv2.ScheduledDepartureDate.str[0:2].astype(int)
    time = dfv2['ActualWheelsOff'].str[0:2].astype(int)
    temp_array = dfv2[['FlightNumber']] * 0
    temp_array = temp_array.rename(columns = {'FlightNumber': 'Temperature'})
    
    #data averaged from https://www.usclimatedata.com/climate/winters/california/united-states/usca1252
    #low temperature == AM, high temperature == PM
    #spring, PM
    temp_array.loc[dfv2[((month == 3) | (month == 4) | (month == 5)) & ((time < 6) | (time >= 18))].index] = sp_l + 459.67
    #spring, AM
    temp_array.loc[dfv2[((month == 3) | (month == 4) | (month == 5)) & ((time >= 6) | (time < 18))].index] = sp_h + 459.67
    #summer, PM
    temp_array.loc[dfv2[((month == 6) | (month == 7) | (month == 8)) & ((time < 6) | (time >= 18))].index] = su_l + 459.67
    #summer, AM
    temp_array.loc[dfv2[((month == 6) | (month == 7) | (month == 8)) & ((time >= 6) | (time < 18))].index] = su_h + 459.67
    #fall, PM
    temp_array.loc[dfv2[((month == 9) | (month == 10) | (month == 11)) & ((time < 6) | (time >= 18))].index] = fa_l + 459.67
    #fal, AM
    temp_array.loc[dfv2[((month == 9) | (month == 10) | (month == 11)) & ((time >= 6) | (time < 18))].index] = fa_h + 459.67
    #winter, PM
    temp_array.loc[dfv2[((month == 12) | (month == 1) | (month == 2)) & ((time < 6) | (time >= 18))].index] = wi_l + 459.67
    #winter, AM 
    temp_array.loc[dfv2[((month == 12) | (month == 1) | (month == 2)) & ((time >= 6) | (time < 18))].index] = wi_h + 459.67
    dfv2['Temperature'] = temp_array
    dfv2['Pressure'] = [29.92/2.036]*len(dfv2) #standard pressure at sea level (29.92 in Hg)
    print('Temperature and pressure coefficients successfully calculated!...')
    
    #begin final emission calculations
    temp_pres_coeff = ((dfv2['Temperature'] / 518.67)**3.5) / (dfv2['Pressure'] / 14.696) #temp and pressure coefficient 
    print(np.mean(temp_pres_coeff))
    
    approach_landing = 0.5
    CCD_time = dfv2['ActualAirborneTime'] - 0.7 - 2.2 - 4 #CCD_time = total_airborne_time - takeoff - climbout - approach
    total_taxi_idle = dfv2['TaxiInTime'] - approach_landing + dfv2['TaxiOutTime'] #this taxi data includes any delays/idle on tarmac, 0.5 mins subtracted for land on runway (counted as approach for ICAO)
    
    taxi_in_time = dfv2['TaxiInTime'] - approach_landing
    taxi_out_time = dfv2['TaxiOutTime']
    
    dfv2['Total Taxi/Idle (min)'] = total_taxi_idle
    dfv2['Cruise Time (min)'] = CCD_time
    
    #Fuel Consumption (fc) Calculations 
    fc_to = temp_pres_coeff * dfv2['Fuel Flow T/O (kg/sec)'] * 0.7 * 60 #get units in seconds, not minutes
    fc_co = temp_pres_coeff * dfv2['Fuel Flow C/O (kg/sec)'] * 2.2 * 60
    fc_ccd = dfv2['Fuel Flow CCD (kg/sec)'] * CCD_time * 60
    fc_app = temp_pres_coeff * dfv2['Fuel Flow App (kg/sec)'] * 4 * 60
    fc_idle = temp_pres_coeff * dfv2['Fuel Flow Idle (kg/sec)'] * total_taxi_idle * 60
    
    fc_taxi_in = temp_pres_coeff * dfv2['Fuel Flow Idle (kg/sec)'] * taxi_in_time * 60 #arriving airports
    fc_taxi_out = temp_pres_coeff * dfv2['Fuel Flow Idle (kg/sec)'] * taxi_out_time * 60 #departing airports
    
    if 'Frac in CA' in list(dfv2): #if the flights are out-of-CA
        #na = non-adjusted, i.e., includes fuel flow and LTO durations for out of CA airports for T/O, C/O and approach
        fc_to_na = fc_to.copy(); fc_to *= dfv2['From CA']
        print(fc_to_na)
        fc_co_na = fc_co.copy(); fc_co *= dfv2['From CA']
        fc_ccd *= dfv2['Frac in CA']
        fc_app_na = fc_app.copy(); fc_app *= (1 - dfv2['From CA'])
        #for taxi/idle:
        taxi_time_adjusted = (dfv2['TaxiOutTime'] * dfv2['From CA']) + (dfv2['TaxiInTime'] * (1 - dfv2['From CA']))
        fc_idle = temp_pres_coeff * taxi_time_adjusted * dfv2['Fuel Flow Idle (kg/sec)'] * 60 #switch to seconds
        
        #also, export the distance flown within CA for calculations later:
        dfv2['Distance (NM)'] = dfv2['Frac in CA'] * dfv2['Distance (NM)']
                
    fuel_consumption = fc_ccd + fc_to + fc_co + fc_app + fc_idle #total fuel consumption
    dfv2['Fuel Consumption (kg)'] = dfv2['No. Engines'] * fuel_consumption;
    #note: cruising EI data already given with pressure/temp coeff multiplied, so no need to multiply
    #by temp and pressure coefficient
        
    #PM Calculation (if the pollutant is SN == 'Smoke Number', the emissions indices are not directly given, 
    #but calculated using the following equation (PM_ei):
    PM_ei = lambda SN: (6.32e-2 * SN + 8.17e-3 * SN**2 + 3.01e-4 * SN**3 + 4.05e-6 * SN**4) / 1.225 #divide by ambient air density
    PM_ei_to = (PM_ei(dfv2['SN EI T/O']) * 50) / 1000 #divide by 1000 to convert mg to grams, so that
    #PM EI is in g/kg fuel burned
    PM_ei_co = (PM_ei(dfv2['SN EI C/O']) * 60) / 1000
    PM_ei_ccd = dfv2['PM EI CCD (g/kg)'] #PM data given directly in EEA Master Emissions Data, not need to use 
    #the above lambda function to calculate it
    PM_ei_app = (PM_ei(dfv2['SN EI App']) * 100) / 1000
    PM_ei_idle = (PM_ei(dfv2['SN EI Idle']) * 120) / 1000
    emissions_PM_total = PM_ei_to * fc_to + PM_ei_co * fc_co + PM_ei_ccd * fc_ccd + PM_ei_app * fc_app + PM_ei_idle * fc_idle
    if 'Frac in CA' in list(dfv2):
        #PM at departing airport only includes fuel flow/times from takeoff, climbout, and taxi/idle at departure airport
        dep_airport_PM = PM_ei_to * fc_to_na + PM_ei_co * fc_co_na + PM_ei_idle * fc_taxi_out
        #PM at arriving airport incl. emissions from approach and taxi/idle at arriving airport
        arr_airport_PM = PM_ei_app * fc_app_na + PM_ei_idle * fc_taxi_in
        dfv2['PM Dep Airport (kg)'] = (dep_airport_PM * dfv2['No. Engines']) / 1000
        dfv2['PM Arr Airport (kg)'] = (arr_airport_PM * dfv2['No. Engines']) / 1000
    else:
        #PM at departing airport only includes emissions from takeoff, climbout, and taxi/idle at departure airport
        dep_airport_PM = PM_ei_to * fc_to + PM_ei_co * fc_co + PM_ei_idle * fc_taxi_out
        #PM at arriving airport incl. emissions from approach and taxi/idle at arriving airport
        arr_airport_PM = PM_ei_app * fc_app + PM_ei_idle * fc_taxi_in
        dfv2['PM Dep Airport (kg)'] = (dep_airport_PM * dfv2['No. Engines']) / 1000
        dfv2['PM Arr Airport (kg)'] = (arr_airport_PM * dfv2['No. Engines']) / 1000           
            
    dfv2['PM (kg)'] = (emissions_PM_total * dfv2['No. Engines']) / 1000
    
    #CO2 calculations:
    CO2_emissions = (fuel_consumption - fc_ccd ) * 3.16 + fc_ccd * dfv2['CO2 EI CCD (kg/kg)']
    dfv2['CO2 (kg)'] = CO2_emissions * dfv2['No. Engines']
    
    #HC, CO, NOx calclations:
    pollutants = ['HC', 'CO', 'NOx']
    for pollutant in pollutants:
        ei_columns = [name for name in list(dfv2) if pollutant in name]
        ei_columns = sorted(ei_columns) #when in alphabetical order: App, C/O, CCD, Idle, T/O
        to_emissions = dfv2[ei_columns[4]] * fc_to
        co_emissions = dfv2[ei_columns[1]] * fc_co
        ccd_emissions = dfv2[ei_columns[2]] * fc_ccd
        app_emissions = dfv2[ei_columns[0]] * fc_app
        idle_emissions = dfv2[ei_columns[3]] * fc_idle
        emissions_total = to_emissions + co_emissions + ccd_emissions + app_emissions + idle_emissions
        if (pollutant == 'NOx') & ('Frac in CA' in list(dfv2)):
            dep_airport_nox = dfv2[ei_columns[4]] * fc_to_na + dfv2[ei_columns[1]] * fc_co_na + dfv2[ei_columns[3]] * fc_taxi_out
            arr_airport_nox = dfv2[ei_columns[0]] * fc_app_na + dfv2[ei_columns[3]] * fc_taxi_in
            dfv2['NOx Dep Airport (kg)'] = (dep_airport_nox * dfv2['No. Engines']) / 1000
            dfv2['NOx Arr Airport (kg)'] = (arr_airport_nox * dfv2['No. Engines']) / 1000
        elif pollutant == 'NOx':
            dep_airport_nox = dfv2[ei_columns[4]] * fc_to + dfv2[ei_columns[1]] * fc_co + dfv2[ei_columns[3]] * fc_taxi_out
            arr_airport_nox = dfv2[ei_columns[0]] * fc_app + dfv2[ei_columns[3]] * fc_taxi_in
            dfv2['NOx Dep Airport (kg)'] = (dep_airport_nox * dfv2['No. Engines']) / 1000
            dfv2['NOx Arr Airport (kg)'] = (arr_airport_nox * dfv2['No. Engines']) / 1000
        column_name = pollutant + ' (kg)'
        dfv2[column_name] = (emissions_total * dfv2['No. Engines']) / 1000
        
    fuel_emissions_final = dfv2[['OD Pair Name', 'AircraftType', 'Carrier', 'FlightNumber', 'Departure', 'Arrival', 
                                     'ScheduledDepartureDate', 'ActualWheelsOff', 'ActualWheelsOn', 'Distance (NM)',
                                     'Cruise Time (min)', 'Total Taxi/Idle (min)', 'Fuel Consumption (kg)', 
                                     'CO2 (kg)', 'HC (kg)', 'CO (kg)', 'NOx (kg)', 'NOx Dep Airport (kg)', 'NOx Arr Airport (kg)', 
                                     'PM (kg)', 'PM Dep Airport (kg)', 'PM Arr Airport (kg)']]
                                 
    fuel_emissions_final['FlightNumber'] = fuel_emissions_final['FlightNumber'].astype(int)
    length_orig = len(fuel_emissions_final)
    fuel_emissions_final = fuel_emissions_final.dropna()
    #fuel_emissions_final = fuel_emissions_final.drop_duplicates()
    length_fin = len(fuel_emissions_final)
    final_data_dropoff = round(100 * (length_orig - length_fin)/len(flight_data), 2)
    print('Data lost after dropping NaN: ' + str(final_data_dropoff) + '%')
    return fuel_emissions_final

In [22]:
x = calculate_fuel_emissions_fulldf(intra_CA)
#x.to_csv(all_US_CA_airport_emissions, index = False)

2022-02-20 18:11:33.142422
First merge made and approximations subsituted
Data lost at 1st merge: 9.98%
Approximations for missing engine models made...
Second merge made...
Data lost at 2nd merge: 5.0%
Temperature and pressure coefficients successfully calculated!...
1.0707657199813971
Data lost after dropping NaN: 0.0%


In [23]:
x

,OD Pair Name,AircraftType,Carrier,FlightNumber,Departure,Arrival,ScheduledDepartureDate,ActualWheelsOff,ActualWheelsOn,Distance (NM),...,Fuel Consumption (kg),CO2 (kg),HC (kg),CO (kg),NOx (kg),NOx Dep Airport (kg),NOx Arr Airport (kg),PM (kg),PM Dep Airport (kg),PM Arr Airport (kg)
0,ACVONT,BD700,EJA,100,ONT,ACV,07/20/2019,13:47,15:00,517.909918,...,4759.683037,14998.223390,0.491617,18.039929,58.428146,6.987790,1.700206,1.347576,0.005411,0.000992
1,ACVVNY,BD700,EJA,112,VNY,ACV,07/20/2019,17:44,18:56,485.869997,...,4790.283844,15095.377259,0.512702,19.565433,58.653735,7.313368,1.765321,1.342151,0.005625,0.001034
2,ACVSNA,C680,EJA,361,SNA,ACV,12/25/2019,17:58,19:26,529.471248,...,2679.520867,8442.987150,0.536423,18.376337,40.369658,2.124281,0.683520,0.220030,0.000507,0.000036
3,ACVLAX,C680,EJA,506,LAX,ACV,07/22/2019,14:53,16:08,501.659190,...,2327.723268,7335.037266,0.522760,16.176010,35.226031,2.362673,0.704274,0.184248,0.000566,0.000025
4,ACVOAK,C750,EJA,990,OAK,ACV,05/16/2019,18:00,18:45,214.085139,...,1566.069451,4935.566411,0.864874,4.897994,19.529446,2.072182,0.483379,0.090232,0.000060,0.000003
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
583808,OAKSTS,C750,XOJ,774,OAK,STS,06/03/2019,09:34,09:49,54.905301,...,540.205124,1704.132495,0.562635,3.226157,6.068915,2.062974,0.451651,0.010388,0.000061,0.000003
583809,OAKSJC,C750,XOJ,782,OAK,SJC,06/16/2019,14:29,14:47,25.617374,...,577.854480,1822.528227,0.455912,2.649011,6.692062,1.989041,0.451651,0.009912,0.000060,0.000003
583810,OAKSFO,C750,XOJ,789,OAK,SFO,06/05/2019,12:56,13:08,9.560567,...,429.995705,1357.198185,0.635683,3.622388,4.502377,2.026008,0.544067,0.003907,0.000061,0.000004
583811,OAKSNA,C750,XOJ,792,OAK,SNA,06/16/2019,17:13,18:15,322.400272,...,2092.240244,6594.425395,0.828765,4.685890,27.147772,2.192356,0.433168,0.121488,0.000063,0.000003


In [24]:
x.to_csv('Results/intra-CA-emissions.csv', index = False)

In [25]:
z = calculate_fuel_emissions_fulldf(out_of_CA)

2022-02-20 18:18:23.377882
First merge made and approximations subsituted
Data lost at 1st merge: 1.63%
Approximations for missing engine models made...
Second merge made...
Data lost at 2nd merge: 4.0%
Temperature and pressure coefficients successfully calculated!...
1.0711754486444105
0          33.440495
1          33.440495
4          17.233617
5          33.440495
6          41.240248
             ...    
1315801    18.372584
1315802    18.372584
1315804    40.593856
1315806    13.514418
1315807    32.916355
Length: 1260158, dtype: float64
Data lost after dropping NaN: 0.0%


In [26]:
z

,OD Pair Name,AircraftType,Carrier,FlightNumber,Departure,Arrival,ScheduledDepartureDate,ActualWheelsOff,ActualWheelsOn,Distance (NM),...,Fuel Consumption (kg),CO2 (kg),HC (kg),CO (kg),NOx (kg),NOx Dep Airport (kg),NOx Arr Airport (kg),PM (kg),PM Dep Airport (kg),PM Arr Airport (kg)
0,ABEBUR,G400,EJM,457,ABE,BUR,10/02/2019,09:51,12:09,182.427449,...,1156.068054,3643.038884,0.119426,7.281864,7.173169,4.221434,0.699134,0.133051,0.003454,0.000242
1,ABEBUR,G400,EJM,457,BUR,ABE,10/05/2019,09:27,16:48,182.427449,...,1187.049529,3742.797502,0.200798,8.433952,9.494777,4.221434,0.770210,0.111924,0.003454,0.000271
4,ABELAX,CL600,DCM,2877,ABE,LAX,01/14/2019,07:43,10:12,192.072882,...,1017.128034,3205.240799,0.430437,3.622187,6.829657,1.923185,0.628877,0.172889,0.022948,0.003086
5,ABELAX,G400,EJM,457,ABE,LAX,10/06/2019,08:19,10:51,192.072882,...,1373.653744,4329.639683,0.280892,11.065888,8.107978,4.221434,1.018976,0.145904,0.003454,0.000372
6,ABEOAK,BD700,EJA,100,ABE,OAK,09/22/2019,15:53,18:14,171.470403,...,1467.087334,4623.388699,0.112726,5.889521,14.407340,7.019207,1.709537,0.759897,0.005406,0.001002
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1315801,VNYYIP,CL600,WWI,84,YIP,VNY,09/28/2019,17:22,18:55,182.088273,...,949.873515,2993.019369,0.290413,2.219662,6.518653,1.662014,0.524837,0.161901,0.021615,0.002221
1315802,VNYYIP,CL600,WWI,93,YIP,VNY,10/01/2019,19:05,20:38,182.088273,...,949.873515,2993.019369,0.290413,2.219662,6.518653,1.686281,0.524837,0.161901,0.021793,0.002221
1315804,VNYYKM,BD700,LXJ,90,VNY,YKM,03/16/2019,12:12,14:00,238.748500,...,2347.126809,7397.883874,0.218079,9.424227,28.090365,6.971259,1.558603,0.557446,0.005362,0.000904
1315806,VNYYUM,C680,EJA,551,VNY,YUM,01/21/2019,18:22,20:07,207.796524,...,1352.486443,4262.246639,0.499617,10.995936,21.647504,2.211843,0.595958,0.088364,0.000534,0.000010


In [27]:
z.to_csv('Results/out-of-CA-emissions.csv', index = False)

# Appendix: Unused Methods

### Estimating CCD Fuel Flow and Emissions Indices using Lagrange Polynomials

In [ ]:
takeoff = [100]*len(fuel_flow_emissions)
climbout = [85]*len(fuel_flow_emissions)
approach = [30]*len(fuel_flow_emissions)
taxi_idle = [7]*len(fuel_flow_emissions)
fuel_flow_emissions['Takeoff'] = takeoff; fuel_flow_emissions['Climb Out'] = climbout
fuel_flow_emissions['Approach'] = approach; fuel_flow_emissions['Taxi/Idle'] = taxi_idle

In [ ]:
columns = ['Fuel Flow T/O (kg/sec)','Fuel Flow Idle (kg/sec)',
           'HC EI T/O (kg/sec)','HC EI Idle (kg/sec)',
           'CO EI T/O (kg/sec)','CO EI Idle (kg/sec)',
           'NOx EI T/O (kg/sec)','NOx EI Idle (kg/sec)',
           'SN T/O','SN Idle']

In [ ]:
fuel_flow_emissions.index

In [ ]:
xx = list(fuel_flow_emissions.loc['AE3007A', 'Fuel Flow T/O (kg/sec)': 'Fuel Flow Idle (kg/sec)'])
yy = list(fuel_flow_emissions.loc['AE3007A', 'NOx EI T/O (g/kg)': 'NOx EI Idle (g/kg)'])

In [ ]:
plt.figure(figsize = (10,7))
for ww in fuel_flow_emissions.index:
    xx = list(fuel_flow_emissions.loc[ww, 'Fuel Flow T/O (kg/sec)': 'Fuel Flow Idle (kg/sec)'])
    yy = list(fuel_flow_emissions.loc[ww, 'NOx EI T/O (g/kg)': 'NOx EI Idle (g/kg)'])
    plt.scatter(xx,yy)
    plt.plot(xx, yy)
plt.show()

In [ ]:
#parse out columns of interest
columns_of_interest = ['Fuel Flow T/O (kg/sec)','Fuel Flow Idle (kg/sec)',
           'HC EI T/O (g/kg)','HC EI Idle (g/kg)',
           'CO EI T/O (g/kg)','CO EI Idle (g/kg)',
           'NOx EI T/O (g/kg)','NOx EI Idle (g/kg)',
           'SN EI T/O','SN EI Idle']

#for each type of aircraft, estimate fuel flow and EI for each pollutant
for ii in range(int(len(columns_of_interest)/2)):
    if ii == 0:
        new_name = 'Fuel Flow Cruising'
    else:
        new_name = re.search(r'\w+(?=\sEI)', columns_of_interest[2*ii]).group(0) + ' Cruising'
    new_list = []
    fig = plt.figure(figsize = (15,10))
    for jj in fuel_flow_emissions.index:
        x = list(fuel_flow_emissions.loc[jj, 'Takeoff':'Taxi/Idle'])
        x_new = range(100)
        y = list(fuel_flow_emissions.loc[jj, columns_of_interest[2*ii]:columns_of_interest[2*ii + 1]])
        f = lagrange(x, y)
        plt.plot(x_new, f(x_new), '#42a1f5', zorder = 1) 
        plt.scatter(x, y, c = '#f54296', zorder = 2)
        plt.xlabel('Thrust Setting (%)')
        plt.ylabel(new_name + ' EI (g/kg of fuel)')
        cruise_addition = f(80)
        if cruise_addition < 0:
            slope = (y[2] - y[1]) / (x[2] - x[1])
            b = y[1] - slope * x[1]
            cruise_addition = slope * 80 + b            
        new_list.append(cruise_addition)
    fuel_flow_emissions[new_name] = new_list
    
fuel_flow_emissions = fuel_flow_emissions.drop(['Takeoff', 'Climb Out', 'Approach', 'Taxi/Idle'], axis = 1)

### Webscraping Pressure and Temperature from wunderground.com

In [ ]:
#defining a function that takes date and location and returns wunderground URL for that location and date
def create_url(date, location):
    location_code = 'K' + location
    
    #get departure date, put departure date in same format as Wunderground
    dep_date = datetime.strptime(date, '%m/%d/%Y')
    dt = datetime.strftime(dep_date, '%Y-%m-%d')
    #lookbehind regex to remove 0-padding from month and date
    dt = re.sub('(?<=-)0', '', dt)

    url = 'https://www.wunderground.com/history/daily/' + location_code + '/date/' + dt
    return url

In [ ]:
#defining a function that takes wunderground URL and time of flight, and returns temperature and pressure
def scrape_temp_pres_data(url):
    #setting up headless webscraper
    chrome_options = Options()
    chrome_options.headless = True
    driver = webdriver.Chrome(options = chrome_options)
    driver.get(url)
    element = WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.CSS_SELECTOR, ".mat-cell.cdk-cell.cdk-column-condition.mat-column-condition.ng-star-inserted")))
    page_contents = BeautifulSoup(driver.page_source, 'lxml')
    tables = page_contents.find_all('table')
    
    #get data of interest (pressure, avg temperature) based on time given in flight data
    historical_data = tables[1]
    driver.quit()
    table_rows = historical_data.findAll('tr')
    print(len(table_rows))
    scraped_data = pd.DataFrame()
    for jj in range(2, len(table_rows)): #starts at second row (1st is header)
        table_row = table_rows[jj]
        columns = table_row.findAll('td')
        output_row = []
        for column in columns:
            output_row.append(column.text.strip())
        scraped_data = scraped_data.append(output_row)
    return scraped_data